In [2]:
import math

import mmh3
class CountMin:
    def __init__(self,w,d):
        self.w=w
        self.d=d
        self.sketch=[[0 for i in range(w)] for j in range(d)]
        self.seeds=[1234,561,76,9565]
    def insert(self,key):
        for i in range(self.d):
            hash_val=mmh3.hash(key,seed=self.seeds[i])
            self.sketch[i][hash_val%self.w]+=1
    def query(self,key):
        min_val=-1
        for i in range(self.d):
            hash_val=mmh3.hash(key,seed=self.seeds[i])
            if min_val==-1 or self.sketch[i][hash_val%self.w]<min_val:
                min_val=self.sketch[i][hash_val%self.w]
        return min_val
class BloomFilter:
    def __init__(self,w,d):
        self.w=w
        self.d=d
        self.sketch=[0 for i in range(w)]
        self.seeds=[1234,561,76,9565]
    def insert(self,key):
        for i in range(self.d):
            hash_val=mmh3.hash(key,seed=self.seeds[i])
            self.sketch[hash_val%self.w]=1
    def query(self,key):
        flag=1
        for i in range(self.d):
            hash_val=mmh3.hash(key,seed=self.seeds[i])
            if self.sketch[hash_val%self.w]==0:
                flag=0
                break
        return flag
    def reset(self):
        for i in range(self.w):
            self.sketch[i]=0
class KeywordSketch:
    def __init__(self,w,d):
        self.w=w
        self.d=d
        self.sketch=[[["",0] for i in range(d)] for j in range(w)]
        self.seed=67483
    def insert(self,doc,word):
        hash_val=doc
        bucket_idx=hash_val%self.w
        min_val=-1
        min_idx=-1
        empty_idx=-1
        for i in range(self.d):
            if self.sketch[bucket_idx][i][0]==word:
                self.sketch[bucket_idx][i][1]+=1
                return
            elif self.sketch[bucket_idx][i][0]=="":
                empty_idx=i
            elif min_val==-1 or self.sketch[bucket_idx][i][1]<min_val:
                min_idx=i
        if empty_idx!=-1:
            self.sketch[bucket_idx][empty_idx][0]=word
            self.sketch[bucket_idx][empty_idx][1]=1
            return
        self.sketch[bucket_idx][min_idx][0]=word
        self.sketch[bucket_idx][min_idx][1]=1
    def query(self,doc):
        hash_val=doc
        bucket_idx=hash_val%self.w
        return self.sketch[bucket_idx]
    
        

In [3]:
def query_similarity(keyword_sketch:KeywordSketch,cm:CountMin,doc1,doc2,total_doc):
    wordlst1=keyword_sketch.query(doc1)
    wordlst2=keyword_sketch.query(doc2)
    word_weight1={}
    word_weight2={}
    sum_cnt1=0
    sum_cnt2=0
    min_weight_sum=0
    max_weight_sum=0
    for pair in wordlst1:
        sum_cnt1+=pair[1]
        word_weight1[pair[0]]=pair[1]
    for pair in wordlst2:
        sum_cnt2+=pair[1]
        word_weight2[pair[0]]=pair[1]
    for word in word_weight1:
        idf=math.log2(total_doc/(cm.query(word)+1))
        tf1=word_weight1[word]/sum_cnt1
        w1=idf*tf1
        tf2=word_weight2.get(word,0)/sum_cnt2
        w2=idf*tf2
        min_weight_sum+=min(w1,w2)
        max_weight_sum+=max(w1,w2)
    for word in word_weight2:
        if word in word_weight1:
            continue
        idf=math.log2(total_doc/(cm.query(word)+1))
        tf1=word_weight1.get(word,0)/sum_cnt1
        w1=idf*tf1
        tf2=word_weight2.get(word,0)/sum_cnt2
        w2=idf*tf2
        min_weight_sum+=min(w1,w2)
        max_weight_sum+=max(w1,w2)
    return min_weight_sum/max_weight_sum
    

In [ ]:
import os
all_data=os.listdir("data")
doc_id=0
vis={}
limit=20
word_idf={}
doc_word_cnt=[]
for data in all_data:
    vis.clear()
    word_cnt={}
    data_file=open("data/"+data,"r")
    lines=data_file.readlines()
    for line in lines:
        src,dst=line.split()
        if vis.get(src,0)==0:
            vis[src]=1
            word_cnt[src]=1
            word_idf[src]=word_idf.get(src,0)+1
        else:
            word_cnt[src]+=1
        if vis.get(dst,0)==0:
            vis[dst]=1
            word_cnt[dst]=1
            word_idf[dst]=word_idf.get(dst,0)+1
        else:
            word_cnt[src]+=1
    doc_word_cnt.append(word_cnt)
    doc_id+=1
    print(doc_id+" done")
    if doc_id==limit:
        break

In [ ]:
doc_sim={}
for i in range(doc_id):
    for j in range(i+1,doc_id):
        code=i*(doc_id+1)+j
        min_weight_sum=0
        max_weight_sum=0
        for word in doc_word_cnt[i]:
            idf=math.log2(doc_id/(word_idf[word]+1))
            tf1=doc_word_cnt[i][word]
            w1=idf*tf1
            tf2=doc_word_cnt[j].get(word,0)
            w2=tf2*idf
            min_weight_sum+=min(w1,w2)
            max_weight_sum+=max(w1,w2)
        for word in doc_word_cnt[j]:
            idf=math.log2(doc_id/(word_idf[word]+1))
            tf1=doc_word_cnt[i].get(word,0)
            if tf1>0:
                continue
            w1=idf*tf1
            tf2=doc_word_cnt[j].get(word,0)
            w2=tf2*idf
            min_weight_sum+=min(w1,w2)
            max_weight_sum+=max(w1,w2)
        doc_sim[code]=min_weight_sum/max_weight_sum

In [0]:
import os
all_data=os.listdir("data")
doc_id=0
cm=CountMin(100000,4)
bf=BloomFilter(10000,4)
ks=KeywordSketch(1000,10)
for data in all_data:
    bf.reset()
    data_file=open("data/"+data,"r")
    lines=data_file.readlines()
    for line in lines:
        src,dst=line.split()
        if not bf.query(src):
            bf.insert(src)
            cm.insert(src)
        ks.insert(doc_id,src)
        if not bf.query(dst):
            bf.insert(dst)
            cm.insert(dst)
        ks.insert(doc_id,dst)
    doc_id+=1
    if limit==doc_id:
        break

In [ ]:
are=0
pairs=0
for i in range(0,doc_id):
    for j in range(i+1,doc_id):
        es_sim=query_similarity(ks,cm,i,j,doc_id)
        ac_sim=doc_sim[i*(doc_id+1)+j]
        if(ac_sim==0):
            continue
        are+=abs(es_sim-ac_sim)/ac_sim
        pairs+=1
are/=pairs
print("are: "+str(are))